In [15]:
from module.filepath import *
import numpy as np
import math

In [36]:
def replace(num):
    return int(num.replace(',', ''))

#Ini gak perlu
def generate_nilai(pendaftar,jurusan):
    # Generate 100 nilai SAINTEK

    # I. Nilai Tes Potensi Skolastik, disesuaikan dengan nilain maksimum dan minimum dari rekapitulasi LTMPT
    # 1. Kemampuan Penalaran Umum
    k_penalaran = np.random.randint(98, 891, pendaftar)
    # 2. Kemampuan Kuantitatif
    k_kuantitaif = np.random.randint(244, 924, pendaftar)
    # 3. Pengetahuan & Pemahaman Umum
    pengetahuan_umum = np.random.randint(204, 848, pendaftar)
    # 4. Kemampuan Memahami Bacaan & Menulis
    baca_tulis = np.random.randint(73, 910, pendaftar) 

    # II. Nilai Tes Kompetensi Akademik
    # 1. Matematika Saintek
    matematika_saintek = np.random.randint(219, 1123, pendaftar)
    # 2. Fisika
    fisika = np.random.randint(227, 1070, pendaftar)
    # 3. Kimia
    kimia = np.random.randint(213, 1067, pendaftar)
    # 4. Biologi
    biologi = np.random.randint(237, 1000, pendaftar)

    #Pilihan User
    pilihan_1 = np.random.randint(0, jurusan, pendaftar)
#     pilihan_2 = np.random.randint(0, jurusan, n)

    #Make Dataframe Ranking

    # Buat Dataframe
    newArray = {'k_penalaran': k_penalaran, 'k_kuantitaif': k_kuantitaif, 'pengetahuan_umum':pengetahuan_umum, 
                'baca_tulis':baca_tulis, 'matematika_saintek': matematika_saintek, 'fisika':fisika, 
                'kimia':kimia, 'biologi':biologi}
    data  = pd.DataFrame(newArray)
    data['rerata'] = data.mean(axis=1)
    data['userId'] = data.index
    data['pilihan_1'] = pilihan_1
    
    n = math.floor(0.1 * len(data))
    top_10 = data.nlargest(n, 'rerata').rerata.iloc[-1]
    #teratas_10_persen = data.sort_values('rerata', ascending = False).head(math.floor(0.1 * len(data))).tail(1)['rerata'].values[0]
    rerata_total = np.mean(data.rerata)
    data = data[['userId','rerata','pilihan_1']]
    
    return data,top_10,rerata_total

def get_daya_tampung(data):
    result = []
    
    #get data jurusan
    kuota = load_file('kuota kampus','kuota kampus')
    kuota['Peminat'] = kuota['Peminat'].apply(replace)
    kuota['batas'] = 500 #Nilai minimal

    
    #get pendaftar
    data = data.pilihan_1.value_counts()
    data = data.sort_index(axis=0)
    list_jurusan = data.index.tolist()
    jumlah_pendaftar = data.values.tolist()
    
    for i,j in zip(list_jurusan,jumlah_pendaftar):
        for k,l in kuota.iterrows():                
            if k == i:
                daya_tampung = l['Daya Tampung']
                #if peminat 0
                if j >= l.Peminat | l.Peminat == 0:
                    daya_tampung_new = daya_tampung
                
                #normal
                else:          
                    daya_tampung_new = math.floor((j/l.Peminat)*daya_tampung)
                    
                #if treshold 0
                if daya_tampung_new == 0:
                    daya_tampung_new = daya_tampung
                
                result.append(daya_tampung_new)
                
    df = pd.DataFrame(list(zip(list_jurusan,result,
                               kuota.batas.tolist())), 
                      columns =['jurusanId','dayatampungBaru','nilaiMinimal'])
    
    return df

def peringkat(rerata,teratas_10_persen,rerata_total):
    if rerata > teratas_10_persen:
        return "Peringkat 10 persen pendaftar"
    elif rerata > rerata_total:
        return "Nilai di atas rata-rata pendaftar"
    else:
        return "Nilai di bawah rata-rata pendaftar"
    
def ranking_pilihan(data_total,data_kuota):
    result = {}
    #Get Jurusan ID
    jurusan = data_kuota.jurusanId.to_list()
    
    #Each Jurusan ID
    for i in jurusan:
        list_user = []
        
        #Get Treshold
        minim = data_kuota.loc[data_kuota.jurusanId==i].nilaiMinimal.values[0]
        kuota = data_kuota.loc[data_kuota.jurusanId==i].dayatampungBaru.values[0]
        
        #Get User by Pilihan
        user = data_total.loc[data_total.pilihan_1==i]
        
        #Sort by rerata
        user = user.sort_values('rerata', ascending = False)
        
        #if surpass treshold
        for index,j in user.iterrows():
            if j.rerata > minim:
                list_user.append(j.userId)
                result[i] = list_user[:kuota] #Limit by kuota
                
    return result